# FSynth Playground

We start with a few prerequisites.

In [1]:
import enum
import heapq
import sys
import string
import random

## FSynth

### Status codes

In [2]:
class Status(enum.Enum):
    Complete = 0
    Incomplete = 1
    Incorrect = -1

### Insertion Alphabet

In [3]:
CHARACTERS = string.printable

### Insertion Index

This is used to indicate whether insertion should be tried anywhere in the prefix or only in the last index.

In [4]:
LAST_INSERT_ONLY = True

### MAX_SIMULTANIOUS_CORRECTIONS

We will have a number of simultaneous plausible fixes at each step. Unfortunately, as the number of such plausible fixes increase, the performance decreases. Hence, with this, we can restrict such simultaneous threads to the best N. The priorities are determined based on (1) the smallest number of repairs (2) maximum boundary (advancement) possible.

In [5]:
MAX_SIMULTANIOUS_CORRECTIONS = 5 # set it to a positive number to restrict the queue.

In [6]:
def filter_best(items):
    if MAX_SIMULTANIOUS_CORRECTIONS < 0: return items
    boundaries = sorted({i.boundary for i in items}, reverse=True)
    return [i for i in items if i.boundary in  boundaries[:MAX_SIMULTANIOUS_CORRECTIONS]]

### MAX_NUM_PER_MASK

This setting controls the number of samples that we evaluate per a given mask.

In [7]:
# the incomplete substring is one behind boundary. i.e inputval[:boundary] 
MAX_NUM_PER_MASK = 1

In [8]:
# at the boundary, it is always wrong.

# We need to sample from inserts and modifiers to prevent them growing
# out of bounds. The idea is to collect all delete,insert,modification indexes
# and form a mask. i.e 3D_4I_5M means that at boundary 3, deletion happened,
# then, in the resulting string, at boundary4, insertion happenbed, and in the
# resuting, at boundary 5, modification happened. Then, we sample from the
# items with same mask. This needs to be done before extending. That is, each
# extend, we should test all possible character extensions on the sampled
# strings.

def sample_items_by_mask(items):
    # sample here. We only want a fixed number of items per mask.
    masks = {}
    for i in items:
        key = (i.mask, i.boundary, i.inputstr[i.boundary-1:i.boundary])
        if i.mask not in masks: masks[key] = []
        masks[key].append(i)

    sampled = []
    for key in masks:
        if len(masks[key]) < MAX_NUM_PER_MASK:
            res = masks[key]
        else:
            res = random.sample(masks[key], MAX_NUM_PER_MASK)
        sampled.extend(res)
    return filter_best(sampled)

### FSynth Repair

In [9]:
class Repair:
    def __repr__(self):
        s = repr(str(self))
        v = (self.inputstr, self.boundary, s)
        return repr(v)

    def set_boundary(self, b):
        assert b >= 0
        self.boundary = b

    def __str__(self):
        return self.inputstr[:self.boundary]

    def __init__(self, inputstr, boundary, mask='', extended=False):
        assert boundary >= 0
        self.inputstr, self.boundary = inputstr, boundary
        self.extended = extended
        self.mask = mask
        self._status = None

    def test(self, mystr):
        return validate(mystr)

    def status(self):
        if self._status is not None: return self._status
        self._status = self.my_status()
        return self._status

    def my_status(self):
        my_str = self.inputstr[:self.boundary]
        if self.test(my_str)[0] == Status.Incorrect: return Status.Incorrect
        if self.test(my_str)[0] == Status.Incomplete: return Status.Incomplete
        # verify this is actually complete. For example, given [*1], [] is not
        # complete. It is likely incorrect. The other chance is 12*45 where
        # 123 is (it is not complete!) incomplete rather than incorrect. To
        # check this, we need to check the next index too.
        if self.boundary >= len(self.inputstr): return Status.Complete
        my_str = self.inputstr[:self.boundary + 1]
        if self.test(my_str)[0] == Status.Incorrect: return Status.Incorrect
        if self.test(my_str)[0] == Status.Incomplete: return Status.Incomplete

        # because if 123*5 is fixed to 12395 is complete, then 1239 is
        # incomplete
        return Status.Incomplete

    def is_incomplete(self):
        return self.status() == Status.Incomplete

    def is_incorrect(self):
        return self.status() == Status.Incorrect

    def is_complete(self):
        return self.status() == Status.Complete

    def apply_delete(self):
        return Repair(self.inputstr[:self.boundary] +
                      self.inputstr[self.boundary + 1:], self.boundary,
                      mask='%s_D%d' % (self.mask, self.boundary)).extend_deleted_item()

    def insert_at(self, k, i, suffix):
        v = self.inputstr[:k] + i + self.inputstr[k:self.boundary] + suffix
        new_item = Repair(v, k, mask='%s_I%d' % (self.mask, k))
        ie = new_item.extend_inserted_item()
        if ie.boundary > k:
            return ie
        return None

    # one of the problems with deletion is that we do not know exactly where the
    # character was deleted from, eventhough the parser can be conforming. For
    # example, given an original string "[1,2,3]", where the corruption happened
    # at the first character, we will only get the failure at the last
    # character. Hence to be accurate, what we shouuld do is to try insert all
    # characters everywhere, and see if it helps. This is ofcourse not a very
    # easy task. So we control this behavior with a switch.
    def insert_char(self, i):
        suffix = self.inputstr[self.boundary:]
        return_lst = []
        if LAST_INSERT_ONLY:
            v = self.insert_at(self.boundary, i, suffix)
            if v is not None: return_lst.append(v)
        else:
            # the repair could be any where from 0 to self.boundary (inclusive).
            # So we try all
            for k in range(self.boundary):
                v = self.insert_at(k,i, suffix)
                if v is not None: return_lst.append(v)
        return return_lst

    def apply_insert(self):
        new_items = []
        for i in CHARACTERS:
            items = self.insert_char(i)
            if items:
                new_items.extend(items)
        return new_items

    def bsearch_extend_item(self):
        bs = binary_search(self.inputstr, left=self.boundary, check=check_is_incomplete)
        assert bs >= 0
        if bs >= len(self.inputstr):
            self.set_boundary(bs)
            self.extended = True
            return self
        # e = self.inputstr[bs] # error causing char.
        self.set_boundary(bs)
        return self

    # there are many more invalid inserts than valid inserts. So searching the
    # whole file again is not useful.
    def lsearch_extend_item(self, nxt=1):
        # need to be done on the item becauese of invariant.
        while True:
            # assert boundary+nxt <= len(inputstr) <- inserts can overshoot
            if (self.boundary + nxt) > len(self.inputstr):
                assert len(self.inputstr) == (self.boundary + nxt - 1)
                self.set_boundary (self.boundary + nxt - 1)
                self.extended = True
                return self
            s = Repair(self.inputstr, self.boundary + nxt)
            if s.is_incomplete():
                #return self.extend_deleted_item()
                nxt += 1
                continue
            if s.is_incorrect():
                # the current nxt is bad, so go back to previous
                self.set_boundary(self.boundary + nxt - 1)
                self.extended = True
                return self
            if s.is_complete():
                self.set_boundary(self.boundary + nxt)
                self.extended = True
                return self
            assert False
        assert False

    def extend_deleted_item(self):
        assert self._status is None
        assert not self.extended
        return self.bsearch_extend_item()
        #return self.lsearch_extend_item(nxt=0)

    def extend_inserted_item(self):
        assert self._status is None
        assert not self.extended
        return self.lsearch_extend_item(nxt=1)
        #return self.bsearch_extend_item()

    def repair_and_extend(self):
        e_arr = []
        item_d = self.apply_delete()
        e_arr.append(item_d)

        # for insert only append if it resulted in a boundary increase
        new_items = self.apply_insert()
        e_arr.extend(new_items)
        return e_arr

### Binary Search

In [10]:
def binary_search(array, left = 0, right = None, check=None):
    if not array: return left
    left, right = 0, len(array) - 1

    #if not check(array, left):
    #    return left
    assert check(array, left)

    if check(array, right):
        return len(array)-1
    # Main loop which narrows our search range.
    while left + 1 < right:
        middle = (left + right) // 2
        if check(array, middle):
            left = middle
        else:
            right = middle
    return left

In [11]:
FLAG=None

In [12]:
def find_fixes(inputval, boundary):
    global FLAG
    # First start with zero edit distance
    # priority, item where item is an array of elements
    next_items = [Repair(inputval, boundary, extended=True)]
    edit_dist = 0
    while True:
        FLAG = edit_dist
        # fetch the first rank groups.
        current_items = next_items
        next_items = []
        chosen_items = sample_items_by_mask(current_items)
        completed = []
        for item in chosen_items:
            # try repair and extending each item until we get incorrect.
            new_items = item.repair_and_extend()

            for i in new_items:
                next_items.append(i)
                if i.is_complete():
                    completed.append(i)
                    yield i
        if completed:
            break
        edit_dist += 1
    assert False

In [13]:
def check_is_incomplete(sval, i):
    s_ = Repair(sval, i)
    s = str(s_)
    return s_.is_incomplete()

In [14]:
def repair(inputval):
    assert check_is_incomplete(inputval, 0) # 1
    # assert not check_is_incomplete(inputval, len(inputval))
    # first do binary search to find the boundary
    # not a requirement. Extend item will do as well.
    boundary = binary_search(inputval, check=check_is_incomplete)
    c = inputval[boundary] # this should be the error causing char.
    assert check_is_incomplete(inputval, boundary)
    # assert not check_is_incomplete(inputval, boundary+1)
    return find_fixes(inputval, boundary)

In [15]:
TESTED = {}

def validate(input_str):
    if input_str in TESTED: return TESTED[input_str]
    TESTED[input_str] = validate_json(input_str)
    return TESTED[input_str]

## Conforming JSON Parser

In [16]:
import json

TESTED = {}
num_runs: int = 0

def logit(*v):
    print(FLAG, *v)
    return

# check if jstr fits in this context.
def it_fits(input_str):
    try:
        json.loads(input_str)
        logit('*', repr(input_str))
        return True
    except Exception as e:
        msg = str(e)
        if msg.startswith('Expecting'):
            # Expecting value: line 1 column 4 (char 3)
            n = int(msg.rstrip(')').split()[-1])
            if n >= len(input_str):
                logit('+', repr(input_str))
                return True
        return False


def validate_json(input_str):
    global num_runs
    num_runs += 1
    try:
        json.loads(input_str)
        logit('*', repr(input_str))
        return Status.Complete, -1, ''
    except Exception as e:
        msg = str(e)
        if msg.startswith('Expecting'):
            # Expecting value: line 1 column 4 (char 3)
            n = int(msg.rstrip(')').split()[-1])
            # If the error is 'outside' the string, it can still be valid
            if n >= len(input_str):
                logit('+', repr(input_str))
                return Status.Incomplete, n, ''
            elif len(input_str) > 1 and input_str[-1] == '.' and input_str[-2].isdigit():
                # JSON returns incorrect for [3. rather than incomplete.
                return Status.Incomplete, n, ''
            else:
                logit('X', repr(input_str))
                remaining = input_str[n:]
                if remaining in ['t', 'tr', 'tru']:
                    # check if it fits first.
                    if it_fits(input_str[:n] + 'true'):
                        return Status.Incomplete, n, input_str[n]
                    return Status.Incorrect, n, input_str[n]
                if remaining in ['f', 'fa', 'fal', 'fals']:
                    if it_fits(input_str[:n] + 'false'):
                        return Status.Incomplete, n, input_str[n]
                    return Status.Incorrect, n, input_str[n]
                if remaining in ['n', 'nu', 'nul']:
                    if it_fits(input_str[:n] + 'null'):
                        return Status.Incomplete, n, input_str[n]
                    return Status.Incorrect, n, input_str[n]
                return Status.Incorrect, n, input_str[n]
        elif msg.startswith('Unterminated'):
            # Unterminated string starting at: line 1 column 1 (char 0)
            n = int(msg.rstrip(')').split()[-1])
            if n >= len(input_str):
                logit('+', repr(input_str))
                return Status.Incomplete, n, ''
            else:
                logit('+', repr(input_str))
                return Status.Incomplete, n, input_str[n]
        elif msg.startswith('Extra data'):
            n = int(msg.rstrip(')').split()[-1])
            if n >= len(input_str):
                logit('X', repr(input_str))
                return Status.Incorrect, n, ''
            else:
                logit('X', repr(input_str))
                return Status.Incorrect, n, input_str[n]
        elif msg.startswith('Invalid '):
            idx = msg.find('(char ')
            eidx = msg.find(')')
            s = msg[idx + 6:eidx]
            n = int(s)
            logit('X', repr(input_str))
            return Status.Incorrect, n, input_str[n]
        else:
            raise e

## Evaluation

FSynth relies on the ability to quickly verify a very large number of results. Unfortunately, this makes usual set of characters unsuitable for Python over WASM as it introduces a few orders of magnitude performance penalty. Hence, we limit our characters.

In [17]:
CHARACTERS = ['[', ']', '{', '}', '"', ',', '.'] + [i for i in string.digits]

In [18]:
def fsynth(inputval):
    fixes = []
    for fix in repair(inputval):
        fixes.append(fix)
        break  # Return only the first fix
    for fix in fixes:
        print('FIXED', repr(str(fix)))
    print(f"Number of oracle runs required for fixing this input: {num_runs}")

In [19]:
inputstr = '[*,+]'
fsynth(inputstr)

None + ''
None X '[*,+'
None X '[*'
None + '['
0 X '[,+'
0 X '[,'
0 + '[['
0 X '[[*'
0 * '[]'
0 X '[]*'
0 + '[{'
0 X '[{*'
0 X '[}'
0 + '["'
0 + '["*'
0 + '["*,'
0 + '["*,+'
0 + '["*,+]'
0 X '[.'
0 + '[0'
0 X '[0*'
0 + '[1'
0 X '[1*'
0 + '[2'
0 X '[2*'
0 + '[3'
0 X '[3*'
0 + '[4'
0 X '[4*'
0 + '[5'
0 X '[5*'
0 + '[6'
0 X '[6*'
0 + '[7'
0 X '[7*'
0 + '[8'
0 X '[8*'
0 + '[9'
0 X '[9*'
1 X '[+'
1 X '[[,'
1 X '[],'
1 X '[{,'
1 + '[",'
1 + '[",+'
1 + '[",+]'
1 + '[0,'
1 X '[0,+'
1 + '[1,'
1 X '[1,+'
1 + '[2,'
1 X '[2,+'
1 + '[3,'
1 X '[3,+'
1 + '[4,'
1 X '[4,+'
1 + '[5,'
1 X '[5,+'
1 + '[6,'
1 X '[6,+'
1 + '[7,'
1 X '[7,+'
1 + '[8,'
1 X '[8,+'
1 + '[9,'
1 X '[9,+'
1 X '[[,+'
1 + '[[['
1 X '[[[*'
1 + '[[]'
1 X '[[]*'
1 + '[[{'
1 X '[[{*'
1 X '[[}'
1 + '[["'
1 + '[["*'
1 + '[["*,'
1 + '[["*,+'
1 + '[["*,+]'
1 X '[[.'
1 + '[[0'
1 X '[[0*'
1 + '[[1'
1 X '[[1*'
1 + '[[2'
1 X '[[2*'
1 + '[[3'
1 X '[[3*'
1 + '[[4'
1 X '[[4*'
1 + '[[5'
1 X '[[5*'
1 + '[[6'
1 X '[[6*'
1 + '[[7'
1 X '[[7*'
1 + '[[8'
